# ETAPA DE TRANSFORMAÇÃO + MINERAÇÃO

### 1. Carregar o dataset tratado

In [ ]:
import pandas as pd

# CARREGAR O ARQUIVO TRATADO
dados = pd.read_excel("dataset_tratado.xlsx")

print("Dataset carregado!")
dados.head()


### 2. Criar atributos temporais
- Datas são úteis para análise
- Criamos colunas novas como mês, ano, semana

In [ ]:
# Mesmo que tenhamos feito antes, ao salvar e carregar de novo (em CSV),
# o Pandas transforma a coluna 'Data' em texto (string).
# Precisamos garantir que ela seja do tipo datetime para trabalhar com datas.
dados["Data"] = pd.to_datetime(dados["Data"])

print(f"Dataset carregado com {len(dados)} linhas.")
print("-" * 40)

# EXTRAÇÃO DE ATRIBUTOS DE TEMPO
print("2. Criando Atributos de Tempo...")

# CRIAÇÃO DE ATRIBUTOS DE TEMPO
dados['Ano'] = dados['Data'].dt.year
dados['Mes'] = dados['Data'].dt.month
dados['Dia_do_Mes'] = dados['Data'].dt.day
dados['Dia_Semana_Nome'] = dados['Data'].dt.day_name(locale='pt_BR') # Nome do dia da semana (ex: 'Sábado')
dados['Semana_Ano'] = dados['Data'].dt.isocalendar().week.astype(int) # Número da semana no ano

display(dados.head())
print("-" * 40)

### 3. Engenharia de Atributos Financeiros
Criando o Ticket Médio por Item/Transação. O "Ticket Médio" é o valor total de venda. Vamos garantir que temos o cálculo correto (embora o seu dataset já tenha o Total de Vendas).

In [ ]:
# CRIAÇÃO DO TICKET MÉDIO DA TRANSAÇÃO
print("3. Criando Atributo de Valor Financeiro (Ticket Médio/Rentabilidade)...")

# A coluna 'Total de Vendas' já é o Ticket Médio da transação.
# Para fins de checagem e garantir que todos os dados fazem sentido:
dados['Total_Vendas_Calculado'] = dados['Quantidade Vendida'] * dados['Preço Unitário']

# Verificamos a diferença entre o valor registrado e o calculado.
# Essa diferença deve ser zero ou muito próxima de zero se os dados estiverem perfeitos.
dados['Diferenca_Vendas'] = dados['Total de Vendas'] - dados['Total_Vendas_Calculado']

# O 'Total de Vendas' será nossa principal métrica de valor da transação.
# Vamos criar uma cópia dela com um nome mais simples para análise futura.
dados['Ticket_Transacao'] = dados['Total de Vendas']

dados[['Quantidade Vendida', 'Preço Unitário', 'Total de Vendas', 'Total_Vendas_Calculado', 'Diferenca_Vendas']].head()

display(dados.head())
print("-" * 40)

### 4. Transformação: Codificação (Encoding) de Variáveis Categóricas
Algoritmos de Mineração de Dados (como Regressão e Clustering) só entendem números. Precisamos transformar texto (como Dia_Semana_Nome ou Loja) em números.
Codificação One-Hot Encoding (Variáveis com poucas categorias)
Usamos pd.get_dummies() para variáveis que têm poucos valores únicos, como Loja e Dia_Semana_Nome.

In [ ]:
# 4.1. One-Hot Encoding para Loja e Dia da Semana
print("4.1. Aplicando One-Hot Encoding em 'Loja' e 'Dia_Semana_Nome'...")

# Loja: Cria 3 novas colunas (Loja_Loja1, Loja_Loja2, Loja_Loja3) com 0 ou 1
# Dia: Cria 7 novas colunas (Dia_Domingo, Dia_Segunda, etc.) com 0 ou 1
dados_codificado = pd.get_dummies(dados, columns=['Loja', 'Dia_Semana_Nome'], prefix=['Loja', 'Dia'])

print("Colunas criadas pelo One-Hot Encoding:")
print([col for col in dados_codificado.columns if 'Loja_' in col or 'Dia_' in col])
print("-" * 40)

### 4.2. Codificação por Frequência (Frequency Encoding) (Variáveis com muitas categorias)
Para variáveis como Produto ou Categoria, que podem ter centenas de valores, o One-Hot Encoding cria muitas colunas desnecessárias. Em vez disso, transformamos o texto na frequência com que ele aparece no dataset. 

In [ ]:
# 4.2. Frequency Encoding para Categoria
print("4.2. Aplicando Frequency Encoding em 'Categoria'...")

# 1. Contamos quantas vezes cada Categoria aparece e normalizamos (transformamos em porcentagem)
frequencia_categoria = dados_codificado['Categoria'].value_counts(normalize=True)

# 2. Criamos uma nova coluna mapeando o nome da Categoria pela sua frequência (porcentagem)
dados_codificado['Categoria_Freq'] = dados_codificado['Categoria'].map(frequencia_categoria)

print(f"O produto da categoria '{dados_codificado['Categoria'].iloc[0]}' tem frequência de {dados_codificado['Categoria_Freq'].iloc[0]:.4f}")
print("-" * 40)

In [ ]:
# 5. Finalização da Transformação
print("5. Finalizando e salvando o dataset transformado...")

# A lista de colunas a remover deve ser ajustada:
colunas_a_remover = [
    'Data',                 # Coluna datetime (usada apenas para extração)
    'Produto',              # Coluna de texto (não codificada)
    'Categoria',            # Coluna de texto (mas já criamos a Categoria_Freq)
    'Total_Vendas_Calculado', # Coluna temporária de checagem
    'Diferenca_Vendas'      # Coluna temporária de checagem
    # 'Dia_Semana_Nome' e 'Loja' NÃO ESTÃO MAIS AQUI, pois foram removidas pelo get_dummies()
]

# Usamos errors='ignore' para garantir que o código não quebre se, por acaso, alguma coluna já tiver sido removida.
dados_finais = dados_codificado.drop(columns=colunas_a_remover, errors='ignore')

# Vamos salvar o dataset final para a Mineração
dados_finais.to_csv("dataset_transformado.csv", index=False)

print("\n-------------------------------------------")
print("✅ FASE DE TRANSFORMAÇÃO CONCLUÍDA")
print(f"Dataset final para Mineração tem {len(dados_finais.columns)} colunas.")
print("Arquivo 'dataset_transformado.csv' salvo com sucesso.")
print("-------------------------------------------")
dados_finais.head()